In [2]:
import os
import cv2
import argparse
import numpy as np
import matplotlib.pyplot as plt
from imutils import paths
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

  
import os
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
import shutil

physical_devices = tf.config.list_physical_devices("GPU")
print("Num GPUs Available: ", physical_devices, len(tf.config.list_physical_devices('GPU')))
# tf.test.gpu_device_name()
device_lib.list_local_devices()

Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')] 1


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12652538658899580371,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 5748293632
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 912110427178217356
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6"]

In [4]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Average

import matplotlib.pyplot as plt
from tensorflow.keras import layers


initial_lr = 1e-3 # learning rate
epochs = 40 # no of eopches to train
batch_size = 32 # batch size 
img_size = 224


# result_path  = ''
result_path  = 'savemodelandresult\\'
model_name1 = 'vgg16_correct\\model-VGG16-31-0.9400'
model_name2 = 'inception_correct\\model-InceptionV3-34-0.9325'
file_format ='.h5'



model_1 = load_model(result_path+model_name1+file_format)
model_2 = load_model(result_path+model_name2+file_format)

model_1 = Model(inputs=model_1.inputs,
                outputs=model_1.outputs,
                name='vgg16')

model_2 = Model(inputs=model_2.inputs,
                outputs=model_2.outputs,
                name='inceptionv3')

models = [model_1,model_2]
model_input = Input(shape=(img_size,img_size,3))
model_output = [model(model_input) for model in models]
ensemble_output = Average()(model_output)
ensemble_model = Model(inputs=model_input,
                        outputs=ensemble_output,
                        name ='ensemble')


#compile our model
print("compiling model.....")
opt = Adam(learning_rate=initial_lr, decay=initial_lr / epochs)
ensemble_model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

ensemble_model.summary()




compiling model.....
Model: "ensemble"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vgg16 (Functional)              (None, 2)            15242050    input_1[0][0]                    
__________________________________________________________________________________________________
inceptionv3 (Functional)        (None, 2)            23903010    input_1[0][0]                    
__________________________________________________________________________________________________
average (Average)               (None, 2)            0           vgg16[0][0]                      
                                                                 incep

In [7]:
result_path1  = 'savemodelandresult\\vgg16_correct\\'
model_name = 'model-VGG16-31-0.9400'




testY_data = np.load(result_path1 + 'testY_' + model_name + '.npy')
trainY_data = np.load(result_path1 + 'trainY_' + model_name + '.npy')
testX_data = np.load(result_path1 + 'testX_' + model_name + '.npy')
trainX_data = np.load(result_path1 + 'trainX_' + model_name + '.npy')


data_generator_with_aug = ImageDataGenerator(
#  rotation_range=45,
#  fill_mode="nearest", 
#  horizontal_flip=True, 
 width_shift_range = 0.1, 
 height_shift_range = 0.1, 
#  shear_range=16
)


In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint

model_filepath = "model-ensemble-{epoch:02d}-{val_accuracy:.4f}.h5"
checkpoint = ModelCheckpoint(
	filepath=model_filepath,
	monitor = 'val_accuracy',
	mode='max',
	save_best_only=True,
	verbose=1
)

#train the head of the network
print("tranning head.........")


history = ensemble_model.fit(
	# x=trainX, y=trainY, batch_size= batch_size,
	data_generator_with_aug.flow(trainX_data, trainY_data, batch_size= batch_size),
	steps_per_epoch=len(trainX_data) // batch_size,
	validation_data = (testX_data, testY_data),
	validation_steps = len(testX_data) // batch_size,
	epochs = epochs,
	# callbacks=[checkpoint]
	)

tranning head.........
Epoch 1/40
50/50 [==============================] - 12s 225ms/step - loss: 0.1557 - accuracy: 0.9500 - val_loss: 0.1898 - val_accuracy: 0.9575
Epoch 2/40
50/50 [==============================] - 11s 217ms/step - loss: 0.1500 - accuracy: 0.9488 - val_loss: 0.1536 - val_accuracy: 0.9550
Epoch 3/40
50/50 [==============================] - 11s 216ms/step - loss: 0.1516 - accuracy: 0.9500 - val_loss: 0.1412 - val_accuracy: 0.9575
Epoch 4/40
50/50 [==============================] - 11s 217ms/step - loss: 0.1404 - accuracy: 0.9469 - val_loss: 0.1449 - val_accuracy: 0.9525
Epoch 5/40
50/50 [==============================] - 11s 215ms/step - loss: 0.1452 - accuracy: 0.9488 - val_loss: 0.2562 - val_accuracy: 0.8775
Epoch 6/40
50/50 [==============================] - 11s 218ms/step - loss: 0.1543 - accuracy: 0.9481 - val_loss: 0.1893 - val_accuracy: 0.9300
Epoch 7/40
50/50 [==============================] - 11s 215ms/step - loss: 0.1445 - accuracy: 0.9463 - val_loss: 0.1371